In [65]:
import requests
import json
import pandas as pd

import plotly.express as px

In [77]:

def get_response(url, endpoint, payload):
    r = requests.get(f'{url}/{endpoint}/?params={payload}')
    js = r.json()
    return js

params = [
    ('gas_boiler', 'natural gas', 0.9, 0.9, False),
    ('ashp', 'electricity', 1.7, 1.7, 3.5),
    ('vrf', 'electricity', 3.2, 2.5, 5.5)
]

paramlist = [{
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': 'ApartmentHighRise',
            'area': 155000 - 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': 'STD2013'
        },
        {
            'type': 'OfficeMedium',
            'area': 15000,
            'heating_fuel': a[1],
            'dhw_fuel': a[1],
            'heating_cop': a[2],
            'dhw_cop': a[3],
            'cooling_cop': a[4],
            'ashrae_standard': 'STD2013'
        },        
    ]
} for a in params]

projections = []
enduses = []
for i, p in enumerate(paramlist):
    herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'
    localhost = 'http://127.0.0.1:5000'
    endpoint = 'get_projection_from_reference_buildings'
    payload = json.dumps(p)

    r = get_response(localhost, endpoint, payload)
    df = pd.DataFrame(r['emissions_projection'])
    df['name'] = params[i][0]
    enduses.append((params[i][0], r['enduses']))
    projections.append(df)
    
proj_df = pd.concat(projections)
proj_piv = pd.pivot_table(proj_df, columns='name', values='kg_co2_per_sf', index='year')
fig = px.line(
    proj_piv, 
    width=900, 
    height=400, 
    template='simple_white'
    )

fig.update_layout({
    'yaxis': {'title': 'kgco2e/sf/yr'},
    'title': {'text': 'emissions standard (kg CO2e/sf/yr),<br>NYU Rubin Hall Alts (from ref buildings)', 'x': 0.45}
})
fig.show()


enduse_dict = {e[0]: e[1]['enduses_per_sf'] for e in enduses}
enduse_df_list = []
for k, v in enduse_dict.items():
    df = pd.DataFrame(v)
    df['name'] = k
    enduse_df_list.append(df)

enduse_df = pd.concat(enduse_df_list)
enduse_df['cat'] = enduse_df['enduse'] + '-' + enduse_df['subcategory']
enduse_df_piv = pd.pivot_table(enduse_df, columns='name', values='kbtu_per_sf', index='enduse')

fig = px.bar(
    enduse_df_piv.sort_values(by=enduse_df_piv.columns[0], ascending=False), 
    barmode='group', template='simple_white', 
    width=900, 
    height=500, 
    title = 'enduses by case, ref-building generated')
fig.show()


In [82]:
# manual enduses

config = {
    'state': 'MA',
    'projection_case': 'MidCase',
    'area': 215000,
    'enduses': [
        {
            'enduse': 'elec',
            'fuel': 'electricity',
            'kbtu': 7129e3
        },
        {
            'enduse': 'gas',
            'fuel': 'natural gas',
            'kbtu': 3111e3
        },
    ]
}

payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{localhost}/get_projection_from_manual_enduses/?params={payload}'
)

js = r.json()
df_projection = pd.DataFrame(js['emissions_projection'])
df_enduses = pd.DataFrame(js['enduses']['enduses_per_sf'])
df_enduses
px.pie(df_enduses, values='kbtu', names='fuel', labels='kbtu', width=250, height=250, hole=0.5, title='site energy').show()
px.line(df_projection, x='year', y='kg_co2_per_sf', width=500,height=300).show()